In [1]:
import cv2
import torch
import pandas as pd
import os
from ultralytics import YOLO

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = YOLO("yolov8n.pt").to(device)

recordings_path = "recordings"  # Исходные видео
timestamps_path = "timestamps"  # Куда сохранять результаты

frame_rate = 15  # ФПС видео
frame_interval = 5  # Запись раз в 0.33 секунд

In [ ]:
print("Processing all videos...")

for root, _, files in os.walk(recordings_path):
    for file in files:
        if file.endswith(".mp4"): 
            video_path = os.path.join(root, file)

            relative_path = os.path.relpath(root, recordings_path)
            output_folder = os.path.join(timestamps_path, relative_path)
            os.makedirs(output_folder, exist_ok=True)

            # Пути для сохранения результатов
            output_csv = os.path.join(output_folder, f"{os.path.splitext(file)[0]}.csv")
            output_txt = os.path.join(output_folder, f"{os.path.splitext(file)[0]}.txt")

            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            results_list = []

            print(f"Processing: {video_path}")

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                if frame_count % frame_interval == 0:
                    results = model(frame, verbose=False)

                    people_count = sum(1 for obj in results[0].boxes.cls if obj == 0)  # Класс "человек" = 0

                    time_sec = frame_count / frame_rate
                    results_list.append([frame_count, time_sec, people_count])

                frame_count += 1

            cap.release()

            # Сохраняем в CSV
            df = pd.DataFrame(results_list, columns=["Frame", "Time (s)", "People Count"])
            df.to_csv(output_csv, index=False)

            # Сохраняем в TXT
            with open(output_txt, "w") as f:
                for row in results_list:
                    f.write(f"{row[0]} {row[1]:.2f} {row[2]}\n")

            print(f"Saved: {output_csv}, {output_txt}")

print("Processing complete!")

Processing all videos...
Processing: recordings/camera4/camera4_20250317_094552.mp4
Saved: timestamps/camera4/camera4_20250317_094552.csv, timestamps/camera4/camera4_20250317_094552.txt
Processing: recordings/camera4/camera4_20250317_043858.mp4
Saved: timestamps/camera4/camera4_20250317_043858.csv, timestamps/camera4/camera4_20250317_043858.txt
Processing: recordings/camera4/camera4_20250317_064546.mp4
